# Nortal - Data Engineer Assignment

## Mapping Data

Our client sent to us his data in a csv format and we need to map this data to our internal
schema. In this task, we expect that you load the csv file, rename the columns according to the
mapping below and cast the columns for the correct format. In addition to the mapping:

- Timestamps are in GMT+3, convert it to GMT+1
- Generate an id column to the items table by concatenating the transaction id column with
the product id column, separated by a dash (-). Ex.: 64532-676

In [7]:
import pandas as pd

In [8]:
items = pd.read_csv('dataset/items.csv')
items.head(5)

,Transaction ID,Product ID
0,110,1
1,111,1
2,112,2
3,112,3
4,113,1


In [37]:
trx = pd.read_csv('dataset/Transactions.csv')
trx.head(5)

,ID,Timestamp,Guset ID,Amount
0,111,08/24/2021 06:08:08,NaN,1.64
1,112,08/24/2021 06:13:45,22.0,3.27
2,113,08/24/2021 06:15:11,11.0,6.12
3,114,08/24/2021 06:24:30,11.0,15.23
4,115,08/24/2021 06:27:43,33.0,5.41


In [26]:
trx['Timestamp'].dtype

dtype('<M8[ns]')

In [23]:
trx['ID'].dtype

dtype('int64')

### Timestamps are in GMT+3, convert it to GMT+1

In [11]:
from datetime import timezone
import datetime

# Getting the current date
# and time
dt = datetime.datetime.now(timezone.utc)
  
utc_time = dt.replace(tzinfo=timezone.utc)
utc_timestamp = utc_time.timestamp()
  
print(utc_timestamp)

1652050053.658963


In [60]:
# trx['Timestamp_Converted'] = pd.to_datetime(trx['Timestamp'], utc=True)
# trx['Timestamp_Converted'] = pd.to_datetime(trx['Timestamp'])
# trx['Timestamp_Converted'] = trx['Timestamp_Converted'].dt.tz_localize('Europe/London')

# trx['Timestamp_Converted'] = trx['Timestamp_Converted'].dt.tz_convert('Europe/Tallinn')
trx['Timestamp_Converted2'] = trx['Timestamp_Converted'].dt.tz_convert('Europe/London')

trx

,ID,Timestamp,Guset ID,Amount,Timestamp_Converted,Timestamp_Converted2
0,111,08/24/2021 06:08:08,NaN,1.64,2021-08-24 08:08:08+03:00,2021-08-24 06:08:08+01:00
1,112,08/24/2021 06:13:45,22.0,3.27,2021-08-24 08:13:45+03:00,2021-08-24 06:13:45+01:00
2,113,08/24/2021 06:15:11,11.0,6.12,2021-08-24 08:15:11+03:00,2021-08-24 06:15:11+01:00
3,114,08/24/2021 06:24:30,11.0,15.23,2021-08-24 08:24:30+03:00,2021-08-24 06:24:30+01:00
4,115,08/24/2021 06:27:43,33.0,5.41,2021-08-24 08:27:43+03:00,2021-08-24 06:27:43+01:00
5,116,08/24/2021 06:28:43,44.0,9.19,2021-08-24 08:28:43+03:00,2021-08-24 06:28:43+01:00
6,117,08/24/2021 06:31:18,22.0,8.13,2021-08-24 08:31:18+03:00,2021-08-24 06:31:18+01:00
7,118,08/24/2021 06:34:00,11.0,13.96,2021-08-24 08:34:00+03:00,2021-08-24 06:34:00+01:00
8,119,08/24/2021 06:34:19,55.0,2.07,2021-08-24 08:34:19+03:00,2021-08-24 06:34:19+01:00
9,120,08/24/2021 06:35:42,66.0,3.06,2021-08-24 08:35:42+03:00,2021-08-24 06:35:42+01:00
